# Experiments as Iterators:  asyncio in Science

<ul>
<li>Daniel Allan</li>
<li>Thomas Caswell</li>
<li>Kenneth Lauer</li>
</ul>

<p>Brookhaven National Lab</p>
<p>Source: https://github.com/NSLS-II</p>
<p>Project Documentation: https://NSLS-II.github.io</p>
</center>

## Origin of this Project

National Synchrontron Light Source II at Brookhaven National Lab

![NSLS-II](https://www.bnl.gov/ps/images/NSLS2-arial-1080px.jpg)

## NSLS-II

* 60 semi-independent research groups (10 so far)
* Scaling up to 19 Pb/year in "expensive pixels"
* No sacred data formats...
* ... but one validatable, extensible (NoSQL) schema for all:
  * metadata
  * data or *references* to data

## Data Acquisition Software Design Goals

* Integrate with the **scipy stack**.
* Support **streaming** data analysis.
* Capture metadata to record
  * a detailed **snapshot** of the hardware (all experiment state);
  * and the scientist's **intention**, the meaning of the measurements.
* Make datasets **searchable** with rich queries on metadata and data.
* As much as possible, avoid inventing a domain-specific language.

## Layers of Software, from the bottom up

* EPICS: Experimental Physics and Industrial Control System
* ophyd, our device abstraction layer
* bluesky, our experiment specification and execution framework

### EPICS

```python
In [1]: import epics

In [2]: epics.caget('SOME_INSCRUTABLE_DEVICE_ID')
Out[2]: 5.0
```

Old-style data acquisition program:

```python
for i in range(5):
    try:
        epics.caput('MOTOR_ID', i)
        value = epics.caget('DETECTOR_ID')
        # bespoke I/O code
    except:
        # bespoke cleanup to ensure hardware safety
```

Metadata is stuffed into filenames or custom headers.

### ophyd (device abstraction layer)

```python
In [3]: import ophyd

In [4]: motor = ophyd.EpicsMotor('MOTOR_ID', name='motor')

In [5]: motor.read()
Out[5]: {'motor':
            {'value': 5.0,
             'timestamp': 1468325228.751564}}
         
In [6]: motor.set(6.0)
```

Devices are expected to support a common interface: `read`, `set`, `stop`, ....

Our devices talk to EPICS, but yours could talk to LabView, RasberryPi, etc.

Devices provide human-friendly names (good for analysis) and a hierarchical structure.

```python
class MultiAxisMirror(ophyd.Device):
    x = ophyd.Component(ophyd.EpicsMotor, ':X')
    y = ophyd.Component(ophyd.EpicsMotor, ':Y')
    pitch = ophyd.Component(ophyd.EpicsMotor, ':P')


mirror = MultiAxisMirror('SOME_ID', name='mirror')

In [1]: mirror.read()
Out[1]: {'mirror_x': {'value': 1.0, ...},
   ...:  'mirror_y': {'value': 1.5, ...},
   ...:  'mirror_pitch': {'value': 0.3, ...}}
```


### bluesky (experiment specification/execution)

New-style data acquisition program:

```python
from bluesky.plans import (open_run, close_run,
                           abs_set, RunEngine,
                           trigger_and_read)
                           
def plan():
    "scan 'motor' from 1 to 5 while reading 'detector'"
    yield from open_run(some_metadata_dict)
    for i in range(5):
        yield from abs_set(motor, i)
        yield from trigger_and_read([detector])
    yield from close_run()
```

The interpreter-like RunEngine performs I/O, safe hardware cleanup, and more.

```python
RE = RunEngine({})
RE(plan())  # execute
```

### A One-Slide Crash Course in ``yield`` and ``yield from``

In [5]:
def g():
    yield 1
    yield 2
    
a = g()  # a is a 'generator instance'
list(a)

## asyncio

* Introduced in Python 3.4 (see PEP 3156)
* An event loop abstraction and a high-level scheduler based on `yield from`

## Why is it useful to think of an experiment as an iterator?

* The experiment must be given as an *expression*, not a *statement*

## So what comprises a "step"?

* a **command**, given as a string, e.g., ``'set'``
* a target **device**, e.g., ``temperature_controller``
* positional and keyword arguments

Express this as a namedtuple, `Msg`.

```
msg = Msg(command, device, *args, **kwargs)
```

In [ ]:
plan = [Msg('')]

## Three different times metadata can be injected

* before experiments starts, in a global stash
* when an experimental plan is written
* interactively, when an experimental plan is executed

In [12]:
from bluesky import RunEngine, Msg
RE = RunEngine({'user': 'dan'})

In [9]:
plan = [Msg('open_run', plan_name='demo plan', num_readings=1),
        Msg('close_run')]

In [15]:
RE(plan, print, mood='optimistic')

start {'plan_name': 'demo plan', 'plan_type': 'list', 'uid': 'eb5d8aae-16f5-4bef-bac7-ab607b5037ae', 'num_readings': 1, 'scan_id': 2, 'time': 1468246826.637021}
stop {'exit_status': 'success', 'reason': '', 'uid': 'dbb3b54c-cce4-4fb0-9f26-e49e7e3a10a5', 'run_start': 'eb5d8aae-16f5-4bef-bac7-ab607b5037ae', 'time': 1468246826.639693}


['eb5d8aae-16f5-4bef-bac7-ab607b5037ae']

# Document Model

* An "Event" is a group of readings that, for scientific purposes, are synchronous.

## Exception handling

In [17]:
from bluesky.plans import finalize_wrapper

In [ ]:
finalize_wrapper()